In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pdp
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import json
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [11]:
# LOAD DATA
INPUT_DIR  = '../../../input/m5-forecasting-accuracy/'
LOG_NAME = '06_19_00_27_25'
data = pd.read_csv(INPUT_DIR + "sales_train_evaluation.csv")
predicted = np.load(f"../log/{LOG_NAME}/pred_valid.npy")
json_load = json.load(open(f'../log/{LOG_NAME}/params.json', 'r'))

In [12]:
predicted.shape

(1, 28)

In [13]:
predicted

array([[0.3869933 , 0.30941105, 0.29050088, 0.30962878, 0.6092186 ,
        0.77236164, 0.9032367 , 0.48667103, 0.5207325 , 0.4392305 ,
        0.46760967, 0.5802398 , 0.8783247 , 0.80810124, 0.54343337,
        0.4143634 , 0.3175019 , 0.39504406, 0.5421915 , 0.6999573 ,
        0.78920704, 0.45636836, 0.40983236, 0.368776  , 0.31874916,
        0.43270084, 0.72368544, 0.774905  ]], dtype=float32)

In [14]:
# InverseするためのScalerを作る
data = data.drop('d_1792',axis=1)
use_days = json_load['use_days']
train_x = np.array(data.iloc[:, -1 * (28*2 + use_days) : -28*2]).sum(axis=0).reshape(1,-1)
# train_x = np.array(data.iloc[:, -1 * (28*1 + use_days) : -28*1]).sum(axis=0).reshape(1,-1)
mm = MinMaxScaler(feature_range=(0.05, 0.95))
mm.fit(train_x.T)

MinMaxScaler(copy=True, feature_range=(0.05, 0.95))

In [15]:
# 予測値をInverse Transform
predicted = mm.inverse_transform(predicted)

In [16]:
days = 100
sells = np.array(data.iloc[:, -1 * days : -28]).sum(axis=0)
sells.reshape(1,-1).shape

(1, 72)

In [17]:
np.concatenate([sells.reshape(1,-1),predicted],axis=1)

array([[53715.        , 48337.        , 46152.        , 37702.        ,
        36545.        , 36436.        , 40392.        , 50857.        ,
        52081.        , 37328.        , 33981.        , 34068.        ,
        34615.        , 37649.        , 47887.        , 50915.        ,
        37259.        , 36536.        , 38458.        , 41459.        ,
        39325.        , 52322.        , 57218.        , 40562.        ,
        37727.        , 37032.        , 38267.        , 40887.        ,
        52711.        , 51421.        , 42035.        , 40117.        ,
        36606.        , 35009.        , 39652.        , 46181.        ,
        47825.        , 37360.        , 35475.        , 34786.        ,
        34003.        , 45611.        , 53863.        , 46360.        ,
        36041.        , 33857.        , 32359.        , 34681.        ,
        45536.        , 52672.        , 56425.        , 40418.        ,
        39683.        , 39134.        , 38116.        , 43220.  

In [18]:
days = 100
f, ax = plt.subplots(figsize=(30, 10))
sells = np.array(data.iloc[:, -1 * days : -28]).sum(axis=0)
plt.plot(np.concatenate([sells.reshape(1,-1),predicted],axis=1).reshape(-1,))
plt.plot(np.array(data.iloc[:, -1 * days :]).sum(axis=0))
# sns.lineplot(predicted.iloc[i, :].index, predicted.iloc[i,:].values.astype("float32"))
plt.vlines(days-28, 20000, 60000, "red", linestyles='dashed')

In [21]:
np.array(data.iloc[:, -1 * days :]).sum(axis=0)

array([53715, 48337, 46152, 37702, 36545, 36436, 40392, 50857, 52081,
       37328, 33981, 34068, 34615, 37649, 47887, 50915, 37259, 36536,
       38458, 41459, 39325, 52322, 57218, 40562, 37727, 37032, 38267,
       40887, 52711, 51421, 42035, 40117, 36606, 35009, 39652, 46181,
       47825, 37360, 35475, 34786, 34003, 45611, 53863, 46360, 36041,
       33857, 32359, 34681, 45536, 52672, 56425, 40418, 39683, 39134,
       38116, 43220, 56340, 53856, 42427, 39069, 35193, 37529, 41789,
       48362, 51640, 38059, 37570, 35343, 35033, 40517, 48962, 49795,
       38793, 35487, 34445, 34732, 42896, 50429, 53032, 43181, 44314,
       39601, 40763, 43805, 54239, 45609, 46400, 39379, 42248, 40503,
       44073, 54308, 59921, 42362, 38777, 37096, 36963, 42552, 51518,
       54338])